# Toxic Comments Classification

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [41]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

## 1. 데이터 로드

In [20]:
df = pd.read_csv('./data/train.csv')
df.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
id               159571 non-null object
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [22]:
df.shape

(159571, 8)

## 2. 필요한 set과 객체 선언

In [3]:
# 축약어 모음
APPOS = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "i would",
"i'd" : "i had",
"i'll" : "i will",
"i'm" : "i am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "i have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not"
}
# 불용어 모음
STOPWORDS = set(stopwords.words("english"))

lemmatizer = WordNetLemmatizer()
tokenizer = TweetTokenizer()
analyzer = SentimentIntensityAnalyzer()

## 3. 함수 정의

In [24]:
# 여러 분류의 toxic feature를 istoxic으로 변환하는 함수
def cat_istoxic(count):
    if count < 1:
        return 0
    else:
        return 1    
    
def get_istoxic(df):
    df['sum_of_toxic'] = df['toxic'] + df['severe_toxic'] + df['obscene'] + df['threat'] + df['insult'] + df['identity_hate']
    df['is_toxic'] = df['sum_of_toxic'].apply(lambda x: cat_istoxic(x))
    df.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'sum_of_toxic'], axis=1, inplace=True)

    return df

In [4]:
# 단어, 문장 등을 count하여 feature로 넣는 함수 
def get_features_count(df):
    df['count_of_sent'] = df['comment_text'].apply(lambda x: len(re.findall('\n', str(x)))+1)
    df['count_of_word'] = df['comment_text'].apply(lambda x: len(str(x).split()))
    df['count_of_unique_word'] = df['comment_text'].apply(lambda x: len(set(str(x).split())))
    df['count_of_punctuations'] = df['comment_text'].apply(lambda x: len([s for s in str(x) if s in string.punctuation]))
    df['count_of_upper_words'] = df['comment_text'].apply(lambda x: len([s for s in str(x).split() if s.isupper()]))
    df['count_of_stopwords'] = df['comment_text'].apply(lambda x: len([s for s in str(x).lower().split() if s in STOPWORDS]))

    df['unique_word_percent'] = df['count_of_unique_word'] * 100 / df['count_of_word']
    df['punct_percent'] = df['count_of_punctuations'] * 100 / df['count_of_word']

    return df

In [5]:
# 쓸모없는 정보(개행, IP주소, USERNAME)을 제거하는 함수
def clean_useless(comment):
    comment = comment.lower()
    
    # 개행 제거
    comment = re.sub('\\n', " ", comment)
    # IP주소 제거
    comment = re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}","",comment)
    # USERNAME 제거
    comment = re.sub("\[\[.*\]","",comment)
    
    # 토큰화
    words = tokenizer.tokenize(comment)
    
    # 줄임말 풀이
    words = [APPOS[word] if word in APPOS else word for word in words]
    # 줄임말 다시 분리
    sent = " ".join(words)
    words = tokenizer.tokenize(sent)
    # stemming
    words = [lemmatizer.lemmatize(word, "v") for word in words]
    # 불용어 제거
    words = [w for w in words if not w in STOPWORDS]
    
    
    clean_sent = " ".join(words)
    
    return(clean_sent)

In [6]:
# 위의 함수를 적용하여 comment를 clean하게 만드는 함수
def get_clean_comment(df):
    df['comment_text'] = df['comment_text'].apply(lambda x: clean_useless(x))
    
    return df

In [7]:
# 감정분석을 하는 함수
def vader_polarity(sentence, threshold=0.1):
    scores = analyzer.polarity_scores(sentence)
    
    # compound 값에 기반하여 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환 
    # compound 가 0.1 보다 크면 긍정 그렇지 않으면 부정
    agg_score = scores['compound']
    final_sentiment = 1 if agg_score >= threshold else 0
    
    return final_sentiment

In [8]:
# 감정분석을 하여 sent_scores라는 피처를 추가하는 함수 
def get_sent_scores(df):
    df['sent_scores'] = df['comment_text'].apply(lambda x: vader_polarity(x))
    
    return df

In [30]:
# 전처리 종합 함수
def preprocessing(df):
    get_istoxic(df)
    get_features_count(df)
    get_clean_comment(df)
    get_sent_scores(df)
    
    df = df.drop(['id', 'comment_text'], axis=1)
    
    return df

## 4. 데이터 전처리

In [31]:
df = preprocessing(df)
df

,is_toxic,count_of_sent,count_of_word,count_of_unique_word,count_of_punctuations,count_of_upper_words,count_of_stopwords,unique_word_percent,punct_percent,sent_scores
0,0,2,43,41,10,2,18,95.348837,23.255814,1
1,0,1,17,17,12,1,2,100.000000,70.588235,1
2,0,1,42,39,6,0,20,92.857143,14.285714,0
3,0,5,113,82,21,5,56,72.566372,18.584071,1
4,0,1,13,13,5,0,5,100.000000,38.461538,1
5,0,3,13,12,4,0,4,92.307692,30.769231,1
6,1,1,8,8,0,8,4,100.000000,0.000000,0
7,0,1,20,20,4,0,12,100.000000,20.000000,0
8,0,1,83,70,19,1,42,84.337349,22.891566,0
9,0,1,12,12,0,0,8,100.000000,0.000000,0


## 5. 모델링 및 학습

In [32]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE

# feature와 label로 분리
y_labels = df['is_toxic']
X_features = df.drop(['is_toxic'], axis=1)
# train set과 validation set으로 분리
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.2, random_state=0)
# Oversampling
X_train_over, y_train_over = smote.fit_sample(X_train, y_train)

# 하이퍼 파라미터 설정
params = {
    'num_leaves' : [32, 64],
    'max_depth' : [128, 160],
    'min_child_samples' : [60, 100],
    'subsample' : [0.8, 1]
}

# 모델링
lgbm_clf = LGBMClassifier(n_estimators=500)
gridcv = GridSearchCV(lgbm_clf, param_grid=params)

# 학습
gridcv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric="auc", eval_set=[(X_train, y_train), (X_test, y_test)])

# 검증 : 평가지표 ROC-AUC
print(gridcv.best_params_)
lgbm_roc_score = roc_auc_score(y_test, gridcv.predict_proba(X_test)[:, -1], average="macro")
print('ROC AUC : {0:4f}'.format(lgbm_roc_score))

[1]	valid_0's auc: 0.759758	valid_0's binary_logloss: 0.315708	valid_1's auc: 0.753808	valid_1's binary_logloss: 0.320709
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.767723	valid_0's binary_logloss: 0.308056	valid_1's auc: 0.761621	valid_1's binary_logloss: 0.312984
[3]	valid_0's auc: 0.76935	valid_0's binary_logloss: 0.302381	valid_1's auc: 0.762111	valid_1's binary_logloss: 0.307323
[4]	valid_0's auc: 0.769648	valid_0's binary_logloss: 0.29789	valid_1's auc: 0.762377	valid_1's binary_logloss: 0.302909
[5]	valid_0's auc: 0.770449	valid_0's binary_logloss: 0.294245	valid_1's auc: 0.763161	valid_1's binary_logloss: 0.299316
[6]	valid_0's auc: 0.773425	valid_0's binary_logloss: 0.291026	valid_1's auc: 0.765621	valid_1's binary_logloss: 0.296158
[7]	valid_0's auc: 0.77446	valid_0's binary_logloss: 0.288417	valid_1's auc: 0.765989	valid_1's binary_logloss: 0.293662
[8]	valid_0's auc: 0.775557	valid_0's binary_logloss: 0.28623	valid_1's auc: 0.766758	va

[73]	valid_0's auc: 0.800311	valid_0's binary_logloss: 0.266564	valid_1's auc: 0.77584	valid_1's binary_logloss: 0.278642
[74]	valid_0's auc: 0.800429	valid_0's binary_logloss: 0.266506	valid_1's auc: 0.77578	valid_1's binary_logloss: 0.27867
[75]	valid_0's auc: 0.800782	valid_0's binary_logloss: 0.266388	valid_1's auc: 0.77568	valid_1's binary_logloss: 0.278681
[76]	valid_0's auc: 0.801148	valid_0's binary_logloss: 0.266252	valid_1's auc: 0.775578	valid_1's binary_logloss: 0.278681
[77]	valid_0's auc: 0.801466	valid_0's binary_logloss: 0.2661	valid_1's auc: 0.775451	valid_1's binary_logloss: 0.278717
[78]	valid_0's auc: 0.801731	valid_0's binary_logloss: 0.266006	valid_1's auc: 0.77551	valid_1's binary_logloss: 0.278703
[79]	valid_0's auc: 0.802036	valid_0's binary_logloss: 0.265881	valid_1's auc: 0.775548	valid_1's binary_logloss: 0.278689
[80]	valid_0's auc: 0.80217	valid_0's binary_logloss: 0.26579	valid_1's auc: 0.775579	valid_1's binary_logloss: 0.278684
[81]	valid_0's auc: 0.802

[36]	valid_0's auc: 0.788854	valid_0's binary_logloss: 0.2712	valid_1's auc: 0.773381	valid_1's binary_logloss: 0.279731
[37]	valid_0's auc: 0.789168	valid_0's binary_logloss: 0.271066	valid_1's auc: 0.773392	valid_1's binary_logloss: 0.279727
[38]	valid_0's auc: 0.789401	valid_0's binary_logloss: 0.270909	valid_1's auc: 0.773484	valid_1's binary_logloss: 0.279693
[39]	valid_0's auc: 0.789808	valid_0's binary_logloss: 0.270744	valid_1's auc: 0.773524	valid_1's binary_logloss: 0.279642
[40]	valid_0's auc: 0.790016	valid_0's binary_logloss: 0.270621	valid_1's auc: 0.773575	valid_1's binary_logloss: 0.279614
[41]	valid_0's auc: 0.790637	valid_0's binary_logloss: 0.270431	valid_1's auc: 0.773631	valid_1's binary_logloss: 0.27955
[42]	valid_0's auc: 0.791015	valid_0's binary_logloss: 0.270298	valid_1's auc: 0.773754	valid_1's binary_logloss: 0.279523
[43]	valid_0's auc: 0.791492	valid_0's binary_logloss: 0.270128	valid_1's auc: 0.77407	valid_1's binary_logloss: 0.279442
[44]	valid_0's auc: 

[109]	valid_0's auc: 0.808357	valid_0's binary_logloss: 0.263415	valid_1's auc: 0.77548	valid_1's binary_logloss: 0.279101
[110]	valid_0's auc: 0.808747	valid_0's binary_logloss: 0.263298	valid_1's auc: 0.775395	valid_1's binary_logloss: 0.279117
[111]	valid_0's auc: 0.80891	valid_0's binary_logloss: 0.263218	valid_1's auc: 0.775368	valid_1's binary_logloss: 0.279123
[112]	valid_0's auc: 0.808975	valid_0's binary_logloss: 0.263176	valid_1's auc: 0.775378	valid_1's binary_logloss: 0.279129
[113]	valid_0's auc: 0.809185	valid_0's binary_logloss: 0.263083	valid_1's auc: 0.775421	valid_1's binary_logloss: 0.279107
[114]	valid_0's auc: 0.809399	valid_0's binary_logloss: 0.26299	valid_1's auc: 0.77543	valid_1's binary_logloss: 0.279115
[115]	valid_0's auc: 0.80976	valid_0's binary_logloss: 0.262867	valid_1's auc: 0.77547	valid_1's binary_logloss: 0.279092
[116]	valid_0's auc: 0.809934	valid_0's binary_logloss: 0.262798	valid_1's auc: 0.775449	valid_1's binary_logloss: 0.279114
[117]	valid_0'

[49]	valid_0's auc: 0.794178	valid_0's binary_logloss: 0.268826	valid_1's auc: 0.778452	valid_1's binary_logloss: 0.278101
[50]	valid_0's auc: 0.794571	valid_0's binary_logloss: 0.268689	valid_1's auc: 0.778687	valid_1's binary_logloss: 0.278059
[51]	valid_0's auc: 0.79479	valid_0's binary_logloss: 0.268575	valid_1's auc: 0.778828	valid_1's binary_logloss: 0.27802
[52]	valid_0's auc: 0.795025	valid_0's binary_logloss: 0.268477	valid_1's auc: 0.778773	valid_1's binary_logloss: 0.278024
[53]	valid_0's auc: 0.795239	valid_0's binary_logloss: 0.268386	valid_1's auc: 0.778798	valid_1's binary_logloss: 0.27802
[54]	valid_0's auc: 0.795612	valid_0's binary_logloss: 0.268252	valid_1's auc: 0.778735	valid_1's binary_logloss: 0.278015
[55]	valid_0's auc: 0.795846	valid_0's binary_logloss: 0.268158	valid_1's auc: 0.778767	valid_1's binary_logloss: 0.277999
[56]	valid_0's auc: 0.796074	valid_0's binary_logloss: 0.268055	valid_1's auc: 0.778792	valid_1's binary_logloss: 0.27797
[57]	valid_0's auc: 

...

[57]	valid_0's auc: 0.81012	valid_0's binary_logloss: 0.26313	valid_1's auc: 0.775042	valid_1's binary_logloss: 0.27924
[58]	valid_0's auc: 0.810493	valid_0's binary_logloss: 0.262953	valid_1's auc: 0.77507	valid_1's binary_logloss: 0.279243
[59]	valid_0's auc: 0.810769	valid_0's binary_logloss: 0.262823	valid_1's auc: 0.775021	valid_1's binary_logloss: 0.27927
[60]	valid_0's auc: 0.811133	valid_0's binary_logloss: 0.262677	valid_1's auc: 0.774943	valid_1's binary_logloss: 0.279294
[61]	valid_0's auc: 0.811497	valid_0's binary_logloss: 0.26248	valid_1's auc: 0.77497	valid_1's binary_logloss: 0.2793
[62]	valid_0's auc: 0.811946	valid_0's binary_logloss: 0.26231	valid_1's auc: 0.77492	valid_1's binary_logloss: 0.279327
[63]	valid_0's auc: 0.812434	valid_0's binary_logloss: 0.262131	valid_1's auc: 0.774889	valid_1's binary_logloss: 0.279339
[64]	valid_0's auc: 0.812767	valid_0's binary_logloss: 0.261985	valid_1's auc: 0.774794	valid_1's binary_logloss: 0.279345
[65]	valid_0's auc: 0.81334

[41]	valid_0's auc: 0.802235	valid_0's binary_logloss: 0.266498	valid_1's auc: 0.774932	valid_1's binary_logloss: 0.279445
[42]	valid_0's auc: 0.802747	valid_0's binary_logloss: 0.266244	valid_1's auc: 0.775086	valid_1's binary_logloss: 0.279385
[43]	valid_0's auc: 0.803137	valid_0's binary_logloss: 0.266063	valid_1's auc: 0.775108	valid_1's binary_logloss: 0.279344
[44]	valid_0's auc: 0.803796	valid_0's binary_logloss: 0.265825	valid_1's auc: 0.775395	valid_1's binary_logloss: 0.279253
[45]	valid_0's auc: 0.804388	valid_0's binary_logloss: 0.265596	valid_1's auc: 0.775326	valid_1's binary_logloss: 0.279258
[46]	valid_0's auc: 0.804786	valid_0's binary_logloss: 0.265403	valid_1's auc: 0.775367	valid_1's binary_logloss: 0.279253
[47]	valid_0's auc: 0.805215	valid_0's binary_logloss: 0.265208	valid_1's auc: 0.775344	valid_1's binary_logloss: 0.279249
[48]	valid_0's auc: 0.805754	valid_0's binary_logloss: 0.265005	valid_1's auc: 0.775317	valid_1's binary_logloss: 0.279244
[49]	valid_0's a

[29]	valid_0's auc: 0.796894	valid_0's binary_logloss: 0.268661	valid_1's auc: 0.77821	valid_1's binary_logloss: 0.278697
[30]	valid_0's auc: 0.79751	valid_0's binary_logloss: 0.2684	valid_1's auc: 0.778364	valid_1's binary_logloss: 0.27864
[31]	valid_0's auc: 0.797895	valid_0's binary_logloss: 0.268161	valid_1's auc: 0.77853	valid_1's binary_logloss: 0.278553
[32]	valid_0's auc: 0.798314	valid_0's binary_logloss: 0.267919	valid_1's auc: 0.778502	valid_1's binary_logloss: 0.278504
[33]	valid_0's auc: 0.798799	valid_0's binary_logloss: 0.267686	valid_1's auc: 0.778574	valid_1's binary_logloss: 0.278475
[34]	valid_0's auc: 0.79923	valid_0's binary_logloss: 0.267495	valid_1's auc: 0.778845	valid_1's binary_logloss: 0.278394
[35]	valid_0's auc: 0.799509	valid_0's binary_logloss: 0.267283	valid_1's auc: 0.778904	valid_1's binary_logloss: 0.278341
[36]	valid_0's auc: 0.799911	valid_0's binary_logloss: 0.267064	valid_1's auc: 0.779046	valid_1's binary_logloss: 0.278301
[37]	valid_0's auc: 0.8

[23]	valid_0's auc: 0.785223	valid_0's binary_logloss: 0.273298	valid_1's auc: 0.77342	valid_1's binary_logloss: 0.280267
[24]	valid_0's auc: 0.785816	valid_0's binary_logloss: 0.272972	valid_1's auc: 0.77385	valid_1's binary_logloss: 0.280049
[25]	valid_0's auc: 0.786299	valid_0's binary_logloss: 0.272712	valid_1's auc: 0.773951	valid_1's binary_logloss: 0.279933
[26]	valid_0's auc: 0.786594	valid_0's binary_logloss: 0.272478	valid_1's auc: 0.774097	valid_1's binary_logloss: 0.279794
[27]	valid_0's auc: 0.787068	valid_0's binary_logloss: 0.272237	valid_1's auc: 0.774602	valid_1's binary_logloss: 0.279634
[28]	valid_0's auc: 0.787482	valid_0's binary_logloss: 0.272017	valid_1's auc: 0.774893	valid_1's binary_logloss: 0.27951
[29]	valid_0's auc: 0.787821	valid_0's binary_logloss: 0.271825	valid_1's auc: 0.774915	valid_1's binary_logloss: 0.279453
[30]	valid_0's auc: 0.788282	valid_0's binary_logloss: 0.271615	valid_1's auc: 0.775197	valid_1's binary_logloss: 0.279341
[31]	valid_0's auc:

[94]	valid_0's auc: 0.807153	valid_0's binary_logloss: 0.263403	valid_1's auc: 0.779832	valid_1's binary_logloss: 0.277477
[95]	valid_0's auc: 0.807431	valid_0's binary_logloss: 0.263293	valid_1's auc: 0.77991	valid_1's binary_logloss: 0.27742
[96]	valid_0's auc: 0.807537	valid_0's binary_logloss: 0.263241	valid_1's auc: 0.779917	valid_1's binary_logloss: 0.277425
[97]	valid_0's auc: 0.807652	valid_0's binary_logloss: 0.263186	valid_1's auc: 0.779965	valid_1's binary_logloss: 0.277417
[98]	valid_0's auc: 0.807935	valid_0's binary_logloss: 0.263062	valid_1's auc: 0.779985	valid_1's binary_logloss: 0.277422
[99]	valid_0's auc: 0.808189	valid_0's binary_logloss: 0.262962	valid_1's auc: 0.780095	valid_1's binary_logloss: 0.277386
[100]	valid_0's auc: 0.808435	valid_0's binary_logloss: 0.262865	valid_1's auc: 0.780141	valid_1's binary_logloss: 0.27739
[101]	valid_0's auc: 0.808623	valid_0's binary_logloss: 0.262776	valid_1's auc: 0.780041	valid_1's binary_logloss: 0.277433
[102]	valid_0's a

## 6. 모델 내보내기

In [12]:
import pickle
from sklearn.externals import joblib

C:\Users\round\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [34]:
joblib.dump(gridcv, 'model.pkl')

['model.pkl']

## 7. 예측

In [9]:
# 예측을 위한 전처리
def preprocessing_predict(comment):
    df = pd.DataFrame({'comment_text': [comment]})
    
    get_features_count(df)
    get_clean_comment(df)
    get_sent_scores(df)
    
    df = df.drop(['comment_text'], axis=1)
    
    return df

In [14]:
sentence = "I fuck you bitch!\n You suck my dicks. I really hate you. Mother FUcking YOu. Son of the bitch!!!!"
df_test = preprocessing_predict(sentence)
df_test

,count_of_sent,count_of_word,count_of_unique_word,count_of_punctuations,count_of_upper_words,count_of_stopwords,unique_word_percent,punct_percent,sent_scores
0,2,19,18,8,2,7,94.736842,42.105263,0


In [15]:
model = joblib.load('model.pkl') 
model.predict(df_test)[0]

0